In [192]:
# pip install python-dotenv

In [193]:
# !pip install openai

In [194]:
import os
from dotenv import load_dotenv
import openai
import json
import datetime
import sys

In [195]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [196]:
# import pandas as pd
# pd.get_option("display.max_columns")
# pd.get_option("display.max_colwidth")
# pd.set_option('display.max_columns', 50)
# pd.set_option('display.max_colwidth', 100)

In [197]:
# "get_belonging_occupation" function defined as Function Calling
# e.g., industry_answer: [{"job title": "Software Engineer", "industry name": "Technology"}, {"job title": "Data Scientist", "industry name": "Science"}, {"job title": "Front-end engineer", "industry name": "To be predicted"}]

def get_belonging_occupation(jobs):
    d = {"Software Engineer": "Technology", "Data Scientist": "Science"}

    # Receives a list of job titles and returns a dict with the corresponding information for each and the industry. However, if there is no specific definition, "To be predicted" is stored.
    def search_industry(job):
        # print(job)
        # print(d.get(job, "To be predicted"))
        return d.get(job, "To be predicted")

    industry_answer = [
        {"job title": job.strip(), "industry name": search_industry(job.strip())} for job in jobs.split(",")
    ]
    return json.dumps(industry_answer)


def get_industry_explanation(industries):
    d = {
            "Technology": "[Microsoft] is one of the most representative company. The technology industry, often referred to as the tech sector or simply 'tech' encompasses a broad range of companies involved in the design, development, manufacturing, and distribution of technological goods and services. This sector is known for its rapid pace of innovation and its significant impact on other industries and aspects of daily life. ",
            "Science": "[NASA] is one of the most representative company. The science industry, often referred to as the scientific sector or research and development (R&D) sector, encompasses a wide range of fields and activities centered on the systematic pursuit of knowledge and the application of scientific research to develop new products, technologies, or processes. This sector plays a crucial role in advancing knowledge, driving innovation, and solving complex problems across various aspects of human life and the environment."
        }

    # Receives a list of job titles and returns a dict with the corresponding information for each and the industry. However, if there is no specific definition, "To be predicted" is stored.
    def search_explanation(industry):
        # print(industries)
        # print(d.get(industries, "To be predicted"))
        return d.get(industry, "To be predicted")

    industry_explanation = [
        {"industry name": industry.strip(), "industry explanation": search_explanation(industry.strip())} for industry in industries.split(",")
    ]
    return json.dumps(industry_explanation)

- 1st step : Provide definition of functions to be used by GPT

In [198]:
# 1st step : Provide definition of "get_belonging_occupation" function to be used by GPT

functions = [
    # Whether GPT uses this function for questions,
    # Also give information to determine what arguments to use when using it.
    {
        "name": "get_belonging_occupation",
        "description": "Get the name of the industry that corresponds to the given job title.",
        "parameters": {
            "type": "object",
            "properties": {
                # Information about the jobs argument
                "jobs": {
                    "type": "string",
                    "description": "Enter job title. Multiple elements can be entered separated by single-byte commas. e.g., Software Engineer, Data Scientist",
                },
            },
            "required": ["jobs"],
        },
    },

    {
        "name": "get_industry_explanation",
        "description": "Get the explanation of the given industry.",
        "parameters": {
            "type": "object",
            "properties": {
                "industries": {
                    "type": "string",
                    "description": "Enter industry name. Multiple elements can be entered separated by single-byte commas. e.g., Manufacturing, Technology, Science",
                },
            },
            "required": ["industries"],
        },
    }
]

In [199]:
# question = "Please answer industries' name that Software Engineer and Data Scientist belong to."
question = "Please describe industries that Software Engineer and Data Scientist belong to."
# question = "How many countries are there in the world?"

- 2nd step : Pass the function definition and question to GPT to determine which function to call.

In [200]:
# 2nd step : Pass the function definition and question to GPT to determine which function to call.
from openai import OpenAI
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# response = openai.ChatCompletion.create(
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": question},
    ],
    functions=functions,
    function_call="auto", # "auto" or "get_belonging_occupation" or "get_industry_explanation"
)

print(response)
print(response.choices[0].message)
print(response.choices[0].message.content)

ChatCompletion(id='chatcmpl-91XS8kWRqnYEyRtoh3YxvwoJ4DAjR', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "jobs": "Software Engineer, Data Scientist"\n}', name='get_belonging_occupation'), tool_calls=None))], created=1710153948, model='gpt-4-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=23, prompt_tokens=148, total_tokens=171))
ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "jobs": "Software Engineer, Data Scientist"\n}', name='get_belonging_occupation'), tool_calls=None)
None


- 3rd step : Call the function determined to be used by GPT and get the result

In [201]:
def execute_function(client, question, functions, message, response):

    if message.function_call:

        # Function name determined to be used
        function_name = message.function_call.name
        
        # Argument dict determined to be used
        arguments = json.loads(message.function_call.arguments)

        # 3rd step : Call the function determined to be used by GPT and get the result
        if function_name == "get_belonging_occupation":
            function_response = eval(function_name)(jobs=arguments.get("jobs")) # function_name: get_belonging_occupation, arguments: {'jobs': 'Software Engineer, Data Scientist, Front-end engineer'}
        elif function_name == "get_industry_explanation":
            function_response = eval(function_name)(industries=arguments.get("industries"))
        
        print("function_response: " + function_response)


        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "user", "content": question},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
            functions=functions,
            function_call="auto", # "auto" or "get_belonging_occupation" or "get_industry_explanation"
        )

        print(response)
        print(response.choices[0].message)
        print(response.choices[0].message.content)

        return response
    
    else:
        print(response)
        print(response.choices[0].message)
        print(response.choices[0].message.content)
        
        return response

In [202]:
for i in range(10): # max retry count
    if response.choices[0].finish_reason == "function_call":
        print("Retrying...")
        message = response.choices[0].message
        response = execute_function(client, question, functions, message, response)
    else:
        break

Retrying...
function_response: [{"job title": "Software Engineer", "industry name": "Technology"}, {"job title": "Data Scientist", "industry name": "Science"}]
ChatCompletion(id='chatcmpl-91XSAxCcFpn3YudltXMTk6rMGTPbz', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "industries": "Technology, Science"\n}', name='get_industry_explanation'), tool_calls=None))], created=1710153950, model='gpt-4-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=22, prompt_tokens=215, total_tokens=237))
ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "industries": "Technology, Science"\n}', name='get_industry_explanation'), tool_calls=None)
None
Retrying...
function_response: [{"industry name": "Technology", "industry explanation": "[Microsoft] is one of the most representative company